In [1]:
import pandas as pd

In [7]:
train = pd.read_csv("train.csv", index_col = "PassengerId")
test = pd.read_csv("test.csv", index_col= "PassengerId")

print(train.shape)
test.head()

(891, 11)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Preprocessing

In [13]:
train.loc[train["Sex"] == "male", "Sex_encode"] = 0
train.loc[train["Sex"] == "female", "Sex_encode"] = 1

test.loc[test["Sex"] == "male", "Sex_encode"] = 0
test.loc[test["Sex"] == "female", "Sex_encode"] = 1

print(train.shape)
test[["Sex", "Sex_encode"]].head()

(891, 12)


,Sex,Sex_encode
PassengerId,,
892,male,0.0
893,female,1.0
894,male,0.0
895,male,0.0
896,female,1.0


## Fill in missing Fare

In [15]:
train["Fare_fillin"] = train["Fare"]
test["Fare_fillin"] = test["Fare"]

test.loc[test["Fare"].isnull(), "Fare_fillin"] = 0
train.loc[train["Fare"].isnull(), "Fare_fillin"] = 0

test[test["Fare"].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode,Fare_fillin
PassengerId,,,,,,,,,,,,
1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,0.0,0.0


## Encoding Embarked

In [23]:
train["Embarked_C"] = (train["Embarked"] == "C")
train["Embarked_S"] = (train["Embarked"] == "S")
train["Embarked_Q"] = (train["Embarked"] == "Q")
print(train.shape)
train[["Embarked", "Embarked_C" , "Embarked_S", "Embarked_Q"]].head()

test["Embarked_C"] = (test["Embarked"] == "C")
test["Embarked_S"] = (test["Embarked"] == "S")
test["Embarked_Q"] = (test["Embarked"] == "Q")
print(test.shape)

(891, 16)
(418, 15)


## Train

In [24]:
## Needs Features(Pclass, Sex, Fare, Embarked) /Label(Survived)

feature_names = ["Pclass", "Sex_encode", "Fare_fillin", 
                 "Embarked_C", "Embarked_S", "Embarked_Q"]
feature_names

['Pclass',
 'Sex_encode',
 'Fare_fillin',
 'Embarked_C',
 'Embarked_S',
 'Embarked_Q']

In [26]:
X_train = train[feature_names]
print(X_train.shape)
X_train.head()

X_test = test[feature_names]
print(X_test.shape)
X_test.head()

(891, 6)
(418, 6)


,Pclass,Sex_encode,Fare_fillin,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,,,
892,3,0.0,7.8292,False,False,True
893,3,1.0,7.0000,False,True,False
894,2,0.0,9.6875,False,False,True
895,3,0.0,8.6625,False,True,False
896,3,1.0,12.2875,False,True,False


In [27]:
label_name = "Survived"

y_train = train[label_name]
print(y_train.shape)
y_train.head()

(891,)


PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

In [31]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth = 5)
model
# 1) fit -> 2) predict(test)

model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [33]:
predictions = model.predict(X_test)

print(predictions.shape)
predictions[0:5]

(418,)


array([0, 0, 0, 0, 1])

## Submit

In [37]:
submit = pd.read_csv("gender_submission.csv", index_col = "PassengerId")

submit["Survived"] = predictions

print(submit.shape)
submit.head()

(418, 1)


,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [38]:
submit.to_csv("decision_tree.csv")